In [1]:
import re
import os
import time
import asyncio

from tqdm import tqdm

from telethon import TelegramClient, events, errors

import numpy as np
import pandas as pd

In [2]:
#ENVIRONMENTAL VARIABLES
DATASET_PATH = os.getenv("NER_DATASET_PATH")
BOT_USERNAME = os.getenv("BOT_USERNAME")

API_ID = os.getenv("API_ID")
API_HASH = os.getenv("API_HASH")
TELEGRAM_NUMBER = os.getenv('TELEGRAM_NUMBER')
TELEGRAM_PASSWORD = os.getenv('TELEGRAM_PASSWORD')

### Задание 1
- **1.1.1** Задача с точки зрения NLP состоит в обработке и трансформации текстовых данных в нужный (обычно численный) формат для применения методов ML.

- **1.2.1** Задача извлечения сущностей представляет собой распознавание частей текста (скорее всего обработанного), которые представляют собой сущности (что такое сущности определется постановкой задачи), и их классификацию по средством присваивания им метки из заранее определенного набора сущностных меток (например, человек, организация, глагол и тд)

- **1.2.2** Классические методы: метод, основанный на использовании регулярных выражений (работает непосредственно с текстом). Методы основанные на классическом ML (численная обработка не связанная напрямую с нейронными сетями). К классическому ML можно отнести такие подходы как вероятностные и CRF модели.

- **1.3.1** Задачу NER с помощью LLM можно решать с помощью подхода prompt-engineering'a используя разные тактики (CoT, in-context, few-shot learning и тд.); Если позволяют ресурсы можно слегка (LoRA) дообучить модель на конкретной downstream задаче (в данном случае NER)

- **1.4.1** Зависит от постановки задачи. Обычно оценка задачи проводится по классическим метрикам классификации accuracy, precision, recall, f1-score. Если в задаче используются LLM то часто также считется CER схожесть одной сырой строки с другой.  

### Задание 2

In [269]:
def get_dataset(dataset_path: str = DATASET_PATH) -> pd.DataFrame:
    """
    Loads data into dataframe by folder path
    
    Parameters
    ----------
    dataset_path: str
        Path to dataset folder (folder that occurs after unzipping sample data)
    """
    # Creating list to store resulting frame rows
    row_list = []
    # Getting generators to iterate over paths
    raw_docs_path_generator = os.listdir(DATASET_PATH+'raw/ru')
    annotated_answer_path_generator = os.listdir(DATASET_PATH+'annotated/ru')

    for raw_doc_path in raw_docs_path_generator:
        for annotated_answer_path in annotated_answer_path_generator:
            # Get ids to match docs and gold answers  
            raw_doc_id = re.findall(r"[0-9]+", raw_doc_path)[0]
            annotated_answer_id = re.findall(r"[0-9]+", annotated_answer_path)[0]

            if raw_doc_id == annotated_answer_id:
                # Read document path to get raw text
                with open(DATASET_PATH+'raw/ru/'+raw_doc_path, 'r') as file:
                    raw_data_content = file.read()
                # Read golden answer lines to get entities and ground truths
                with open(DATASET_PATH+'annotated/ru/'+annotated_answer_path, 'r') as file:
                    annotated_answer_lines = file.readlines()
                
                # Normalize answer spaces and capitalization
                # Starting from second line to avoid ID line
                # Transform list of lines to a list of word lists
                splitted_answer_lines = [re.split(r"\s+",line.lower()) for line in annotated_answer_lines[1:]]
                empty_removed_answer_lines = [[word for word in line if word != ''] for line in splitted_answer_lines] 
                # Separate with tabs and spaces
                result_list_lines = []
                for one_word_list_line in empty_removed_answer_lines:
                    N = len(one_word_list_line)
                    # Calculate where the tab should be
                    tab_indx = (N-2)//2 
                    # Prapare for regular space separation
                    space_sep_list = [one_word_list_line[:tab_indx],
                                      one_word_list_line[tab_indx:N-2]]
                    # Prepare for tab separation
                    tab_sep = [' '.join(word_list) for word_list in space_sep_list] + one_word_list_line[-2:]
                    result_list_lines.append('\t'.join(tab_sep))
                
                # Join lines with new lines to return normalized golden answer 
                golden_answer = '\n'.join(result_list_lines)
                # Find entities from golden answer to add to row
                entity = re.findall(r"\s[loc|per|org|evt|pro]+\s", golden_answer)
                stripped_entity = [ent.strip() for ent in entity]

                # Creating and appending to row_list for further df creation
                row_to_append = {
                    "document_id": raw_doc_id,
                    "document_text": raw_data_content,
                    "entity": stripped_entity,
                    "golden_answer": golden_answer
                }
                row_list.append(row_to_append)
    return pd.DataFrame(row_list)

In [270]:
dataset = get_dataset()
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   document_id    9 non-null      object
 1   document_text  9 non-null      object
 2   entity         9 non-null      object
 3   golden_answer  9 non-null      object
dtypes: object(4)
memory usage: 420.0+ bytes
None


In [271]:
dataset.head(3)

,document_id,document_text,entity,golden_answer
0,1006,ru-1006\nru\n2018-07-09\nhttp://polit.ru/news/...,"[evt, pro, per, per, loc, loc, per, per, per, ...",brexit\tbrexit\tevt\tevt-brexit\nfacebook\tfac...
1,1003,ru-1003\nru\n2018-07-09\nhttps://echo.msk.ru/n...,"[per, loc, loc, per, per, org, per, org, per, ...",борис джонсон\tборис джонсон\tper\tper-boris-j...
2,1017,ru-1017\nru\n2018-07-09\nhttp://www.unn.com.ua...,"[evt, pro, per, per, per, per, loc, per, per, ...",brexit\tbrexit\tevt\tevt-brexit\nthe guardian\...


### Задание 3

In [ ]:
doc = 'one'
main_prompt = f"""
Задание: Решить задачу NER на экспертном уровне для заданного документа и заданных меток.
Метки: per(люди); org(организации); loc(локации, места); evt(мироприятия, евенты); pro(продукты)
Документ: {doc}
---
Форматирование ответа:
1. Ответ должен содержать только решение для заданной задачи и ничего больше.
2. Решение задачи содержит строки, состоящие из 4-x элементов (имя сущности в документе, начальная форма сущности, метка сущности, идентификатор на английском), разделенных '\t'
3. Если элемент разделяемый, то разделение осуществляется пробелом (дональд трамп, the times, наполеон бонапарт)
4. Строки разделяются '\n'
4. Ответ должен быть приведен к нижнему регистру
---
Примеры:
1. В Пакистане проходят акции исламистов против отмены Верховным судом смертного приговора за богохульство христианке Асии Биби. -> 
пакистане\tпакистан\tloc\tgpe-pakistan\n
верховным судом\tверховный суд\torg\torg-supreme-court-of-pakistan\n
асии биби\tасия биби\tper\tper-asia-bibi
2. В Екатерининском зале Кремля прошла встреча Владимира Путина с участниками Конгресса молодых учёных. ->
екатерининском зале кремля\tекатерининский зал кремля\tloc\tloc-st-catherine-hall\n
владимира путина\tвладимир путин\tper\tper-vladimir-putin\n
конгресса молодых учёных\tконгресс молодых учёных\tevt\tevt-congress-of-young-scientists
3. Дженсен Хуанг представил видеокарту GeForce RTX 50 на GTC 2025 Washington D.C. Keynote. -> 
дженсен хуанг\tдженсен хуанг\t

"""

brexit	brexit	evt	evt-brexit
the guardian	the guardian	pro	pro-the-guardian
борис джонсон	борис джонсон	per	per-boris-johnson
бориса джонсона	борис джонсон	per	per-boris-johnson
бориса джонсона	бориса джонсона	per	per-boris-johnson
бориса	борис	per	per-boris-johnson
великобритании	великобритания	loc	gpe-great-britain
джонсон	джонсон	per	per-boris-johnson
дэвид дэвис	дэвид дэвис	per	per-david-davis
ес	ес	org	org-european-union
киев	киев	loc	gpe-kiev
мид соединенного королевства	мид соединенного королевства	org	org-foreign-office
стив бейкер	стив бейкер	per	per-steve-baker
тереза мэй	тереза мэй	per	per-theresa-may
унн	унн	org	org-unn


In [441]:
some_string = 'so me'
print(some_string)

so me


In [ ]:
def get_prompt(dataframe_row: pd.Series) -> str:
    """
    Given dataframe row returns a prompt for LLM to solve NER task for that row

    Parameters
    ----------
    dataframe_row: pd.Series
        Dataframe row to solve for

    Returns
    -------
    prompt: str
        Prompt for LLM to solve NER for document_text 
    """
    return 0
'--------------------------------------------------------------------------------'

'--------------------------------------------------------------------------------'

### Задание 4

In [3]:
# Initialize the client. 'anon' is the session file name.
client = TelegramClient('session', API_ID, API_HASH)

async def main():
    # Connect the client
    await client.connect()

    # If the user is not authorized, prompt for login
    if not await client.is_user_authorized():
        phone_number = input("Enter your phone number: ")
        await client.send_code_request(phone_number)
        code = input("Enter the code you received: ")
        password = input("Enter your 2FA password: ")
        try:
            await client.sign_in(phone_number, code)
        except errors.SessionPasswordNeededError as error_message:
            print(error_message, end='\n\n')
            await client.sign_in(password=password)

    print("Logged in successfully!")

    # You can now perform actions with the client, e.g., get information about yourself
    me = await client.get_me()
    print(f"Logged in as: {me.first_name}")

    # Disconnect the client when done to save the session
    await client.disconnect()

# Run the asynchronous main function
await main()

Two-steps verification is enabled and a password is required (caused by SignInRequest)

Logged in successfully!
Logged in as: Лучиан


In [4]:
async def ask(message):
    await client.start()
    async with client.conversation(BOT_USERNAME, timeout=60) as conv:
        await conv.send_message(message)
        while True:
            response = await conv.get_response()
            if response.text.split(',')[0] == 'Запрос принят':
                time.sleep(1)
                continue
            else:
                return response.text

In [5]:
request_list = []

In [8]:
for num in range(1,3):
    request_list.append(await ask(f"Посоветуй классическую литературу"))

CancelledError: 

In [9]:
request_list

['−99−100',
 '−98−99',
 'Вот подборка классических произведений русской литературы, ставших неотъемлемой частью культурного наследия нашей страны:\n\n**▌ Русская классика XIX века**\n\n- **Александр Пушкин** — __«Евгений Онегин»__ («роман в стихах»), __«Капитанская дочка»__, __«Борис Годунов»__\n- **Михаил Лермонтов** — __«Герой нашего времени»__, __«Демон»__, __«Мцыри»__\n- **Николай Гоголь** — __«Мертвые души»__, __«Ревизор»__, __«Шинель»__\n- **Иван Тургенев** — __«Отцы и дети»__, __«Дворянское гнездо»__, __«Накануне»__\n- **Федор Достоевский** — __«Преступление и наказание»__, __«Идиот»__, __«Братья Карамазовы»__\n- **Лев Толстой** — __«Война и мир»__, __«Анна Каренина»__, __«Воскресение»__\n- **Антон Чехов** — рассказы (__«Человек в футляре»__, __«Дама с собачкой»__) и пьесы (__«Три сестры»__, __«Вишнёвый сад»__)\n\n**▌ Советская литература XX века**\n\n- **Максим Горький** — __«На дне»__, __«Жизнь Клима Самгина»__\n- **Алексей Толстой** — трилогия __«Хождение по мукам»__, роман _

In [303]:
client.send_code_request(phone='+79258465683')

<coroutine object AuthMethods.send_code_request at 0x703d2e08dd80>

In [301]:
auth = client.is_user_authorized()
auth

<coroutine object UserMethods.is_user_authorized at 0x703d2777ec20>

In [296]:
client.send_message(BOT_USERNAME, 'Привет сколько в тебе параметров')

<coroutine object MessageMethods.send_message at 0x2c09fb70>

In [290]:
client._get_response_message(BOT_USERNAME)

TypeError: MessageParseMethods._get_response_message() missing 2 required positional arguments: 'result' and 'input_chat'

In [284]:
async def main():
    await start_client()
    answer = ask()
    print(f"Answer: {answer}")

In [285]:
asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop